<a href="https://colab.research.google.com/github/sbooeshaghi/azucar/blob/main/analysis/293T/merge_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# this takes 5 minutes
!git clone https://github.com/sbooeshaghi/azucar.git

fatal: destination path 'azucar' already exists and is not an empty directory.


In [7]:
!cat azucar/analysis/293T/obs7/out/x* > azucar/analysis/293T/obs7/out/matrix.mtx.gz

In [8]:
import glob
import os
from collections import defaultdict
from scipy.io import mmread, mmwrite
import json
import pandas as pd

In [9]:
base = "azucar/analysis/293T"
fns = glob.glob(os.path.join(base, "obs*"))


In [10]:
md = defaultdict()
for fn in fns:
  obs = fn.split('/')[-1]
  m = json.load(open(f"{fn}/metadata.json", "r"))
  md[obs] = m
print(json.dumps(md, indent=4, default=str))

{
    "obs2": {
        "cell_origin": "293T",
        "organism": "homo_sapiens",
        "replicate": 1,
        "lane_10x": 1,
        "molecule_type": "tags",
        "observarion": 2
    },
    "obs1": {
        "cell_origin": "293T",
        "organism": "homo_sapiens",
        "replicate": 1,
        "lane_10x": 1,
        "molecule_type": "cdna",
        "observarion": 1
    },
    "obs5": {
        "cell_origin": "293T",
        "organism": "homo_sapiens",
        "replicate": 2,
        "lane_10x": 2,
        "molecule_type": "cdna",
        "observarion": 5
    },
    "obs6": {
        "cell_origin": "293T",
        "organism": "homo_sapiens",
        "replicate": 2,
        "lane_10x": 2,
        "molecule_type": "tags",
        "observarion": 6
    },
    "obs3": {
        "cell_origin": "293T",
        "organism": "homo_sapiens",
        "replicate": 2,
        "lane_10x": 1,
        "molecule_type": "cdna",
        "observarion": 3
    },
    "obs4": {
        "cell_origi

In [13]:
%%time
b = []
mx = []
g = []

for obs, d in md.items():
  if d["molecule_type"] == "cdna":
    out = os.path.join(base, obs, "out/")
    mtx_fn = os.path.join(out, "matrix.mtx.gz")
    bcs_fn = os.path.join(out, "map.txt.gz")
    genes_fn = os.path.join(out, "genes.txt.gz")
    
    # load matrix, barcodes, genes
    mtx = mmread(mtx_fn).tocsr()
    bcs = pd.read_csv(bcs_fn, compression="gzip", sep="\t")
    genes = pd.read_csv(genes_fn, header=None, names=["genes"], compression = "gzip")

    mask = bcs["assignments"].isna().values
    bcs["assignments"][mask] = "unassigned"
    bcs["lane_10x"] = d["lane_10x"]
    bcs["replicate"] = d["replicate"]
    bcs["observation"] = d["observarion"]

    b.append(bcs)
    mx.append(mtx)
    g.append(genes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 3min 25s, sys: 1.58 s, total: 3min 26s
Wall time: 3min 26s


In [14]:
from scipy.sparse import vstack

In [15]:
mtx = vstack([*mx])
bcs = pd.concat(b)

In [16]:
mtx

<25372x60664 sparse matrix of type '<class 'numpy.float64'>'
	with 61573342 stored elements in Compressed Sparse Row format>

In [17]:
groups = sorted(bcs.assignments.unique())

In [19]:
d = {g: {"mtx": None, "bcs": None, "genes": None} for g in groups}
for g in groups:
  mask = bcs.eval(f"assignments == '{g}'").values
  d[g]["mtx"] = mtx[mask]
  d[g]["bcs"] = bcs[mask]
  d[g]["genes"] = genes

In [20]:
for k, v in d.items():
  mmwrite(f"{k}.matrix.mtx", v["mtx"])
  v["bcs"].to_csv(f"{k}.barcodes.txt", sep="\t", index=False, header=False)
  v["genes"].to_csv(f"{k}.genes.txt", sep="\t", index=False, header=False)


In [21]:
!mkdir no_sugar control tmg unassigned

In [22]:
!mv no_sugar.* no_sugar
!mv control.* control
!mv tmg.* tmg
!mv unassigned.* unassigned

In [23]:
!tar -cvf no_sugar.tar.gz no_sugar
!tar -cvf control.tar.gz control
!tar -cvf tmg.tar.gz tmg
!tar -cvf unassigned.tar.gz unassigned

no_sugar/
no_sugar/no_sugar.matrix.mtx
no_sugar/no_sugar.genes.txt
no_sugar/no_sugar.barcodes.txt
control/
control/control.matrix.mtx
control/control.barcodes.txt
control/control.genes.txt
tmg/
tmg/tmg.barcodes.txt
tmg/tmg.genes.txt
tmg/tmg.matrix.mtx
unassigned/
unassigned/unassigned.barcodes.txt
unassigned/unassigned.matrix.mtx
unassigned/unassigned.genes.txt


In [24]:
!ls -lht

total 1.9G
-rw-r--r-- 1 root root 365M Nov 11 05:38 unassigned.tar.gz
-rw-r--r-- 1 root root 425M Nov 11 05:38 tmg.tar.gz
-rw-r--r-- 1 root root 540M Nov 11 05:38 control.tar.gz
-rw-r--r-- 1 root root 597M Nov 11 05:38 no_sugar.tar.gz
drwxr-xr-x 2 root root 4.0K Nov 11 05:38 unassigned
drwxr-xr-x 2 root root 4.0K Nov 11 05:38 tmg
drwxr-xr-x 2 root root 4.0K Nov 11 05:38 control
drwxr-xr-x 2 root root 4.0K Nov 11 05:38 no_sugar
drwxr-xr-x 4 root root 4.0K Nov 11 04:57 azucar
drwxr-xr-x 1 root root 4.0K Nov  1 13:35 sample_data
